In [1]:
import numpy as np
import progressbar 
import random 
import os
import cv2
import h5py

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

import keras
from keras.applications import imagenet_utils
from keras.applications import Xception
from keras.layers import Input
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing.image import img_to_array
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, model_from_json

import matplotlib.pyplot as plt
%matplotlib inline

from helpers import HDF5DatasetWriter
from helpers import Utils

Using TensorFlow backend.


In [2]:
output_path = "datasets/UCF-101-hdf5/fixed-sequance-rgbmean-xception-imagenet-feature.hdf5"

In [3]:
db = h5py.File("datasets/UCF-101-hdf5/fixed-sequance-rgbmean.hdf5")

x = db["images"][:]
y = db["labels"][:]

In [4]:
num_classes = y.shape[1]
num_classes

8

In [5]:
x = pad_sequences(x, maxlen=20, truncating="post")

lb = LabelBinarizer()
y = lb.fit_transform(y)

In [6]:
x.shape

(200, 20, 60, 80, 3)

In [7]:
input_tensor = Input(shape=(60, 80, 3))
cnn_model = Xception(weights='imagenet', include_top=False, input_tensor=input_tensor)
for layer in cnn_model.layers:
    layer.trainable = False
cnn_model_output = cnn_model.output
cnn_bottleneck = GlobalAveragePooling2D()(cnn_model_output)
cnn_bottleneck_model = Model(inputs=cnn_model.input, outputs=cnn_bottleneck)

In [ ]:
image = x[0][1]
image = np.expand_dims(image, axis=0)

In [ ]:
features = cnn_bottleneck_model.predict(image)

In [ ]:
sequences = []
for (j, sequence) in enumerate(x[0:4]):
    print(sequence.shape)
    

In [ ]:
sequences = np.array(sequences)
sequences.shape

In [ ]:
batch_size=4
sequences = []
for i in np.arange(0, x.shape[0], batch_size):
    batchData = x[i:i + batch_size]
    batchLabels = y[i:i + batch_size]
    for (j, sequence) in enumerate(batchData):
        print(j)
        images = []
        for k, image in enumerate(sequence):
            image = np.expand_dims(image, axis=0)
            features = cnn_bottleneck_model.predict(image)
            images.append(features)
        sequences.append(images)

In [ ]:
def extract_feature_data(model, x, y, output_path, batch_size = 32, buffer_size = 1000):
    dataset = HDF5DatasetWriter((x.shape[0], 2048), y.shape, output_path, bufSize = buffer_size)
    
    widgets = ["Extracting Features: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
    pbar = progressbar.ProgressBar(maxval=x.shape[0], widgets=widgets).start()
    
    sequences = []
    for i, sequence in enumerate(x):
        images = []
        for j, image in enumerate(sequence):
            image = np.expand_dims(image, axis=0)
            features = model.predict(image)
            images.append(features)
        sequences.append(images)
        pbar.update(i)
    dataset.add(sequences, y)
    dataset.close()
    pbar.finish()

In [ ]:
extract_feature_data(cnn_bottleneck_model, x, y, output_path, 500)

In [ ]:
import h5py

In [ ]:
db = h5py.File(output_path)

x = db["images"][:]
y = db["labels"][:]

In [ ]:
x.shape

In [ ]:
y.shape